In [1]:
import pandas as pd
import numpy as np
import os
import json
import ast
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import chain
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
english_stop_words = list(stopwords.words('english'))
import gensim.corpora as corpora
import gensim
from gensim.utils import simple_preprocess
from pprint import pprint
import pyLDAvis.gensim
import pickle
import pyLDAvis
import time
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pi96a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = "C:/Drive D/Datasets/Aminer Data"
os.chdir(path)

In [3]:
os.listdir()

['dblp-ref-0.json',
 'dblp-ref-1.json',
 'dblp-ref-2.json',
 'dblp-ref-3.json',
 'new_dblp-ref-0.json',
 'new_dblp-ref-1.json',
 'new_dblp-ref-2.json',
 'new_dblp-ref-3 LDA topic words.csv',
 'new_dblp-ref-3.json']

In [4]:
def convert_file(file):
    with open(file) as json_file:
        data = json_file.readlines()
    
    data = [ast.literal_eval(d.strip("\n").strip("'")) for d in data]
    
    json_filename = "new_{}".format(file)
    if not os.path.isfile("new_{}".format(json_filename)):
        with open(json_filename, "w") as file:
            json.dump(data, file)
    
    print("Completed file : {}".format(file))

In [5]:
#files = [f for f in os.listdir()]
#
#for file in files:
#    convert_file(file)

In [6]:
#%%time
#df1 = pd.read_json('new_dblp-ref-0.json')
#df2 = pd.read_json('new_dblp-ref-1.json')
#df3 = pd.read_json('new_dblp-ref-2.json')
#df4 = pd.read_json('new_dblp-ref-3.json')

In [7]:
#%%time
#df_all = pd.concat([df1, df2, df3, df4], axis = "rows")

In [8]:
%%time
df = pd.read_json("new_dblp-ref-3.json")
# drop all NA values from the dataset
print("Before Dropping NA : ", df.shape)
df = df.dropna().reset_index(drop = True)
print("After Dropping NA : ", df.shape)

Before Dropping NA :  (79007, 8)
After Dropping NA :  (35174, 8)
CPU times: total: 234 ms
Wall time: 1.36 s


In [9]:
df["n_references"] = df["references"].apply(lambda x: len(x))
df["GT_zero_citations"] = df["n_citation"].apply(lambda x: "Yes" if x > 0 else "No")

In [10]:
all_authors = df["authors"].tolist()
flatten_all_authors = list(chain.from_iterable(all_authors))

In [11]:
#Counter(flatten_all_authors)

In [12]:
"""
Things to do:
- Build a content based recommendation system.
- Work on a hybrid approach using topic modelling and content of the papers.
- use authors and topic modelling scores to recommend papers.
"""

'\nThings to do:\n- Build a content based recommendation system.\n- Work on a hybrid approach using topic modelling and content of the papers.\n- use authors and topic modelling scores to recommend papers.\n'

### LDA Model and Utilities

In [13]:
# generating content column for each row in dataset
df["Content"] = df["abstract"].str.lower() + df["title"].str.lower()

In [14]:
%%time
# Creating stop words list
stop_words_extended_list = ["the", "of", "and", "a", "in", "to", "is", 
                            "?", "for", "we", "that", "-", "=", "with", ",", 
                            "on", "as", "by", "are", "this", "be", ".", 
                            "can", "an", "from", ")", "which", "it", "+", 
                            "each", "our", "at", "where", "has", "such", 
                            "a", "b", "c", "d", "e", "f", "g", "h", "i",
                            "j", "k", "l", "m", "n", "o", "p", "q", "r",
                            "s", "t", "u", "v", "w", "x", "y", "z", 
                            "0", "1", "2", "3", "4", "5", "6", "7", "8", 
                            "9", "et", "j.", "", "(", "a)", "b)"]
english_stop_words.extend(stop_words_extended_list)
english_stop_words = set(english_stop_words)

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
%%time
# Tokenizing the documents after preprocessing
full_string_list = []
punctuations = [",", ".", "!", "!","?", "=", "+", "??", ":", "|", "<", ">", "&", "?)", ";", "[", "]", "?.", "al.", "}", "{", "~", "..", "(?", "_", "???"]
for sentence in df["Content"].values:
    temp_string_list = []
    for words in sentence.split(" "):
        if words not in english_stop_words and words not in punctuations:
            temp_string_list.append(words)
    full_string_list.append(temp_string_list)

CPU times: total: 906 ms
Wall time: 2.37 s


In [16]:
%%time
# creating a dictionary from the tokenized documents
id2word = corpora.Dictionary(full_string_list)

CPU times: total: 2.36 s
Wall time: 3.7 s


In [17]:
%%time
# Creating a bag of word representation for each document
texts = full_string_list
corpus = [id2word.doc2bow(text) for text in texts]

CPU times: total: 1.33 s
Wall time: 2.07 s


In [29]:
%%time
# Building an LDA model which has 5 clusters
# number of topics
num_topics = 6

# building an LDA model
lda_model = gensim.models.LdaModel(corpus = corpus, id2word = id2word, num_topics = num_topics)

if not os.path.exists("new_dblp-ref-3 LDA.model"):
    lda_model.save("new_dblp-ref-3 LDA.model")

CPU times: total: 13 s
Wall time: 14.2 s


In [19]:
%%time
# popular topics in LDA model
topics = lda_model.show_topics(num_topics = num_topics, num_words = 80, formatted = False)

CPU times: total: 0 ns
Wall time: 18.5 ms


In [20]:
%%time
# document topics 
document_topics = lda_model[corpus]

CPU times: total: 0 ns
Wall time: 0 ns


In [21]:
%%time
# creating document and topic dictionary
document_topic_dict = dict()
for i, doc_topics in enumerate(document_topics):
    topic_index = []
    topic_probs = []
    for dt in doc_topics:
        topic_index.append(dt[0])
        topic_probs.append(dt[1])
    
    max_prob_index = np.argmax(topic_probs)
    document_topic_dict[f"Document {i + 1}"] = doc_topics[max_prob_index][0]

CPU times: total: 6.8 s
Wall time: 9.77 s


In [22]:
%%time
doc_topic_df = pd.DataFrame(document_topic_dict, index = ["Topic"]).T.reset_index(drop = True)

CPU times: total: 125 ms
Wall time: 212 ms


In [23]:
%%time
topic_words_dictionary = dict()
for index, top_words in topics:
    topic_words_dictionary[index] = []
    for word in top_words:
        topic_words_dictionary[index].append(word[0])

CPU times: total: 0 ns
Wall time: 0 ns


In [24]:
# Dictionary to start with search using words
topic_words_dictionary.keys()

dict_keys([0, 1, 2, 3, 4, 5])

In [25]:
doc_topic_df.value_counts()

Topic
5        11974
1         6035
0         5258
3         5156
4         5094
2         1657
Name: count, dtype: int64

In [26]:
topic_words_df = pd.DataFrame(topic_words_dictionary)
topic_words_df.columns = [f"Topic_{i}" for i in range(6)]

In [27]:
topic_words_df.to_csv("new_dblp-ref-3 LDA topic words.csv")

In [28]:
doc_topic_df.to_csv("new_dblp-ref-3 doc-topic-df.csv")